# Open AI Modellerini İnce Ayar Yapma

Bu not defteri, Open AI tarafından sağlanan [İnce Ayar](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) belgelerindeki mevcut rehbere dayanmaktadır.

İnce ayar, temel modellerin performansını, belirli kullanım durumu veya senaryoya ilişkin ek veri ve bağlamla yeniden eğiterek uygulamanız için iyileştirir. _few shot learning_ ve _retrieval augmented generation_ gibi prompt mühendisliği tekniklerinin, kaliteyi artırmak için varsayılan prompt'u ilgili verilerle geliştirmenize olanak tanıdığını unutmayın. Ancak, bu yaklaşımlar hedeflenen temel modelin maksimum token pencere boyutuyla sınırlıdır.

İnce ayar ile, modeli gerekli verilerle etkili bir şekilde yeniden eğitiyoruz (maksimum token penceresine sığabilecekten çok daha fazla örnek kullanmamıza izin veriyor) - ve çıkarım zamanında örneklerin sağlanmasına artık ihtiyaç duymayan _özel_ bir model sürümünü dağıtıyoruz. Bu, prompt tasarımımızın etkinliğini artırmakla kalmaz (token penceresini diğer şeyler için kullanmada daha fazla esneklik sağlar) aynı zamanda maliyetlerimizi de potansiyel olarak iyileştirir (çıkarım zamanında modele göndermemiz gereken token sayısını azaltarak).

İnce ayar 4 adımdan oluşur:
1. Eğitim verilerini hazırlayın ve yükleyin.
1. İnce ayarlı modeli elde etmek için eğitim işini çalıştırın.
1. İnce ayarlı modeli değerlendirin ve kalite için yineleyin.
1. Memnun kaldığınızda ince ayarlı modeli çıkarım için dağıtın.

Tüm temel modellerin ince ayarı desteklemediğini unutmayın - en güncel bilgi için [OpenAI belgelerini kontrol edin](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst). Ayrıca daha önce ince ayar yapılmış bir modeli de ince ayar yapabilirsiniz. Bu öğreticide, ince ayar için hedef temel model olarak `gpt-35-turbo` kullanacağız.

---


### Adım 1.1: Veri Kümenizi Hazırlayın

Bir element hakkında soruları bir limerik ile yanıtlayarak periyodik tabloyu anlamanıza yardımcı olan bir sohbet botu oluşturalım. _Bu_ basit öğreticide, verilerin beklenen formatını gösteren birkaç örnek yanıtla modeli eğitmek için sadece bir veri kümesi oluşturacağız. Gerçek dünya kullanımında, çok daha fazla örnek içeren bir veri kümesi oluşturmanız gerekir. Ayrıca, uygulama alanınız için mevcutsa açık bir veri kümesini kullanabilir ve ince ayar için yeniden biçimlendirebilirsiniz.

`gpt-35-turbo` üzerine odaklandığımız ve tek turlu yanıt (sohbet tamamlama) aradığımız için, OpenAI sohbet tamamlama gereksinimlerini yansıtan [bu önerilen formatı](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) kullanarak örnekler oluşturabiliriz. Çok turlu sohbet içeriği bekliyorsanız, ince ayar sürecinde hangi mesajların kullanılacağını (veya kullanılmayacağını) belirtmek için `weight` parametresini içeren [çok turlu örnek formatını](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) kullanırsınız.

Burada öğreticimiz için daha basit tek turlu formatı kullanacağız. Veriler, her satırda 1 kayıt bulunan ve her biri JSON formatında nesne olarak temsil edilen [jsonl formatındadır](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst). Aşağıdaki kesit, örnek olarak 2 kayıt göstermektedir - tam örnek seti için [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) dosyasına bakabilirsiniz (10 örnek) ve bunları ince ayar öğreticimizde kullanacağız. **Not:** Her kayıt _tek satırda_ tanımlanmalıdır (biçimlendirilmiş JSON dosyasında olduğu gibi satırlara bölünmemelidir).

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

Gerçek dünya kullanımında iyi sonuçlar için çok daha büyük bir örnek setine ihtiyacınız olacak - burada kalite ile ince ayar için gereken zaman/maliyetler arasında bir denge olacaktır. Süreci göstermek için ince ayarı hızlıca tamamlayabilmek adına küçük bir set kullanıyoruz. Daha karmaşık bir ince ayar öğreticisi için [bu OpenAI Cookbook örneğine](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) bakabilirsiniz.


---

### Adım 1.2 Veri Kümenizi Yükleyin

Verileri, Dosyalar API'sini kullanarak [burada açıklandığı gibi](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file) yükleyin. Bu kodu çalıştırabilmek için öncelikle aşağıdaki adımları tamamlamış olmanız gerekir:
 - `openai` Python paketini yüklediniz (en son özellikler için >=0.28.0 sürümünü kullandığınızdan emin olun)
 - `OPENAI_API_KEY` ortam değişkenini OpenAI API anahtarınız olarak ayarladınız
Daha fazla bilgi için, kurs için sağlanan [Kurulum rehberine](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) bakın.

Şimdi, yerel JSONL dosyanızdan yükleme için bir dosya oluşturmak üzere kodu çalıştırın.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


---

### Adım 2.1: SDK ile İnce Ayar işini oluşturma


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


---

### Adım 2.2: İşin Durumunu Kontrol Etme

`client.fine_tuning.jobs` API'si ile yapabileceğiniz birkaç şey şunlardır:
- `client.fine_tuning.jobs.list(limit=<n>)` - Son n ince ayar işini listele
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Belirli bir ince ayar işinin detaylarını al
- `client.fine_tuning.jobs.cancel(<job_id>)` - Bir ince ayar işini iptal et
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - İşten n olayı listele
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Sürecin ilk adımı, verilerin doğru formatta olduğundan emin olmak için _eğitim dosyasını doğrulamaktır_.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


---

### Adım 2.3: İlerlemenin izlenmesi için olayları takip etme


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Adım 2.4: Durumu OpenAI Kontrol Panelinde Görüntüleyin


Durumu ayrıca OpenAI web sitesini ziyaret ederek ve platformun _İnce Ayar_ bölümünü keşfederek görüntüleyebilirsiniz. Bu, mevcut işin durumunu gösterir ve ayrıca önceki iş yürütme geçmişini takip etmenizi sağlar. Bu ekran görüntüsünde, önceki yürütmenin başarısız olduğunu ve ikinci çalışmanın başarılı olduğunu görebilirsiniz. Bağlam için, bu durum ilk çalışmanın yanlış biçimlendirilmiş kayıtlar içeren bir JSON dosyası kullandığında meydana geldi - düzeltildikten sonra, ikinci çalışma başarıyla tamamlandı ve modeli kullanıma sundu. 

![İnce ayar iş durumu](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba.tr.png)


Durum mesajlarını ve metrikleri, görsel panoda aşağı doğru kaydırarak da görüntüleyebilirsiniz:

| Mesajlar | Metrikler |
|:---|:---|
| ![Messages](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b.tr.png) |  ![Metrics](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a65229.tr.png)|


---

### Adım 3.1: Kimliği Alın ve İnce Ayarlı Modeli Kodda Test Edin


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


---

### Adım 3.2: İnce Ayarlı Modeli Playground'da Yükleyin ve Test Edin

Artık ince ayarlı modeli iki şekilde test edebilirsiniz. İlk olarak, Playground'a gidip Modeller açılır menüsünden yeni ince ayarlı modelinizi listelenen seçenekler arasından seçebilirsiniz. Diğer seçenek ise, yukarıdaki ekran görüntüsünde görülen İnce Ayar panelinde gösterilen "Playground" seçeneğini kullanmaktır; bu, temel ve ince ayarlı model sürümlerini yan yana gösteren aşağıdaki _karşılaştırmalı_ görünümü başlatır ve hızlı değerlendirme yapmanızı sağlar.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016.tr.png)

Eğitim verilerinizde kullanılan sistem bağlamını doldurun ve test sorunuzla devam edin. Her iki tarafın da aynı bağlam ve soruyla güncellendiğini fark edeceksiniz. Karşılaştırmayı çalıştırın ve çıktılar arasındaki farkı göreceksiniz. _İnce ayarlı modelin yanıtı örneklerinizde sağladığınız formatta sunduğunu, temel modelin ise sadece sistem istemini takip ettiğini not edin_.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350.tr.png)

Karşılaştırmanın ayrıca her model için token sayılarını ve çıkarım için geçen süreyi sağladığını fark edeceksiniz. **Bu özel örnek, süreci göstermek için basit bir örnektir ve gerçek bir veri seti veya senaryoyu yansıtmaz**. Her iki örneğin de aynı sayıda token gösterdiğini (sistem bağlamı ve kullanıcı istemi aynıdır) ve ince ayarlı modelin çıkarım için daha fazla zaman aldığını fark edebilirsiniz (özel model).

Gerçek dünya senaryolarında, bu tür oyuncak bir örnek kullanmayacak, gerçek verilerle ince ayar yapacaksınız (örneğin, müşteri hizmetleri için ürün kataloğu) ve yanıt kalitesi çok daha belirgin olacaktır. _Bu_ bağlamda, temel modelle eşdeğer yanıt kalitesi elde etmek, daha fazla özel istem mühendisliği gerektirecek ve bu da token kullanımını ve muhtemelen çıkarım için ilgili işlem süresini artıracaktır. _Bunu denemek için, OpenAI Cookbook'taki ince ayar örneklerine göz atabilirsiniz._

---


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Feragatname**:  
Bu belge, AI çeviri servisi [Co-op Translator](https://github.com/Azure/co-op-translator) kullanılarak çevrilmiştir. Doğruluk için çaba gösterilse de, otomatik çevirilerin hatalar veya yanlışlıklar içerebileceğini lütfen unutmayın. Orijinal belge, kendi dilinde yetkili kaynak olarak kabul edilmelidir. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımı sonucu oluşabilecek yanlış anlamalar veya yorum hatalarından sorumlu değiliz.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
